In [1]:
import numpy as np
import cv2
import os
import moviepy.editor as moviepy

In [2]:
cat=cv2.imread("cat-headphones.png")
cat=cv2.resize(cat,(572,322))
album=cv2.imread("albumkapagi.jpg")


In [3]:
print(album.shape)
u=[0,374,374,0]
v=[0,0,374,374]

(375, 375, 3)


In [4]:
def coefmatrix(u,v,x,y):
    M= [[x[0], y[0], 1,  0,  0,  0,  -x[0]*u[0], - y[0]*u[0]],
    [0,  0,  0,  x[0], y[0], 1,  -x[0]*v[0], - y[0]*v[0]],
    [x[1], y[1], 1,  0,  0,  0,  -x[1]*u[1], -y[1]*u[1]],
    [0,  0,  0,  x[1], y[1], 1,  -x[1]*v[1], -y[1]*v[1]],
    [x[2], y[2], 1,  0,  0,  0,  -x[2]*u[2], -y[2]*u[2]],
    [0,  0,  0,  x[2], y[2], 1,  -x[2]*v[2], -y[2]*v[2]],
    [x[3], y[3], 1,  0,  0,  0,  -x[3]*u[3], -y[3]*u[3]],
    [0,  0,  0,  x[3], y[3], 1,  -x[3]*v[3], -y[3]*v[3]]
    ];
    U=[u[0],v[0],u[1],v[1],u[2],v[2],u[3],v[3]]
    
    a,resid,rank,s = np.linalg.lstsq(M,U)#linalg.solve gave me singular matrix
    #error, so I search online and find this solution
    
    area= abs(x[1]-x[0])*abs(y[1]-y[2])
    return a,area

In [5]:
planes = np.zeros((9,472,4,3))


In [6]:
for i in range(1,10):
    with open("Plane_"+str(i)+".txt") as f:
        content = f.readlines()
        for line_id in range(len(content)):
            sel_line = content[line_id]
            sel_line = sel_line.replace(')\n', '').replace("(", '').split(")")

            for point_id in range(4):
                sel_point = sel_line[point_id].split(" ")

                planes[i-1,line_id,point_id,0] = float(sel_point[0])
                planes[i-1,line_id,point_id,1] = float(sel_point[1])
                planes[i-1,line_id,point_id,2] = float(sel_point[2])


In [7]:
images_list = []

In [8]:
for i in range(472):
    blank_image = np.empty((322,572,3), np.uint8)
    blank_image.fill(255)
    areas = []
    dsts= []
    for j in range(9):

            pts = planes[j,i,:,:].squeeze()[:,0:2].astype(np.int32)

            temp = np.copy(pts[3,:])
            pts[3, :] = pts[2,:]
            pts[2, :] = temp
            pts = pts.reshape((-1, 1, 2))
            
            a,area= coefmatrix(u,v,pts.T.flatten()[0:4],pts.T.flatten()[4:8])
            mat=[[a[0],a[1],a[2]],[a[3],a[4],a[5]],[a[6],a[7],1]]
            mat_inv=np.linalg.pinv(mat)
            
            dst = cv2.warpPerspective(album, mat_inv, (572,322))
            areas.append(area)
            dsts.append(dst)
            cv2.polylines(blank_image, [pts], True, (0, 255, 255))   
    
    for c in range(0,len(areas)):
        for r in range(c+1,len(areas)):
            if(areas[c]==areas[r]):
                areas[c]=areas[c]+1  
    zipped_lists = zip(areas, dsts)
    sorted_pairs = sorted(zipped_lists)

    tuples = zip(*sorted_pairs)
    areas, dsts = [ list(tuple) for tuple in  tuples] #sort destination images according to areas
    
    
            
    for k in range (9):
         
        destination=dsts[k]
        foreground=np.logical_or(destination[:,:,1]>0, destination[:,:,0]>0)
        nonzero_x,nonzero_y =np.nonzero(foreground)
        nonzero_values = destination[nonzero_x,nonzero_y,:]
        blank_image[nonzero_x,nonzero_y,:]=nonzero_values #inspired from part1 your codes
        
        if(k==6): #there is 3 image front 6 image back so I've added cat after 6th step
            foreground=np.logical_or(cat[:,:,1]>0, cat[:,:,0]>0)
            nonzero_x,nonzero_y =np.nonzero(foreground)
            nonzero_values = cat[nonzero_x,nonzero_y,:]
            blank_image[nonzero_x,nonzero_y,:]=nonzero_values #add cat

    blank_image = blank_image[:,:,[2,1,0]]
    images_list.append(blank_image)

In [9]:
clip = moviepy.ImageSequenceClip(images_list, fps = 25)
clip.write_videofile("part3_video.mp4", codec="libx264")

t:   0%|                                                                             | 0/472 [00:00<?, ?it/s, now=None]

Moviepy - Building video part3_video.mp4.
Moviepy - Writing video part3_video.mp4



Moviepy - Done !
Moviepy - video ready part3_video.mp4
